In [ ]:
import sys
!{sys.executable} -m pip install sympy
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install dfply

# Task 1 (40 points):
Based on Planck’s law for the radiance of the Sun, assuming a temperature $T$ of $5775\text{K}$, create four plots, which show
A. The spectral irradiance versus photon wavelength
B. The spectral irradiance versus photon energy
C. The photon flux versus photon wavelength
D. The photon flux versus energy.

You can convert the radiance of the object to the irradiance by multiplying the radiance by the steradian angle of the Sun’s disc as it is observed on Earth. That value is $6.794\cdot10^{-5}\text{sr}$. At the end of Task 1, you should have four plots that look similar to the ones on the PV Lighthouse web site:
https://www2.pvlighthouse.com.au/resources/courses/altermatt/The%20Solar%20Spectrum/Intensity%20or%20photon%20flux.aspx

The axis labels should match the ones on PV Lighthouse, except that you should not convert the photon flux to
$\frac{\text{mA}}{\text{cm}^2\cdot\text{eV}}$, but leave it as $\frac{\text{Number of photons}}{\text{s}\cdot\text{m}^2\cdot\text{eV}}$.

In [139]:
from numpy import linspace, exp, e
import QMFunctions as qm
from QMFunctions import c, q

from plotly.express import scatter
from pandas import DataFrame #easiest way to get plotly to display axis labels is if the data is put in a dataframe with labeled columns

sun_steradian_angle = 6.794*10**(-5) # in sr

T = 5775

A. The spectral irradiance versus photon wavelength

In [36]:
def irradiance_wavelength(Lambda):
  q = qm.q
  h = q*qm.h
  k_b = q*qm.k_b
  #h and k_b multiplied by q = 1.6022*10**(-19) to convert the eV's in their units to Joules, so the irradiance has units of Watts per m^3
  irradiance = (2 * h * (c**2)/((exp(h * c/(Lambda * k_b * T)) - 1) * Lambda**5)) * sun_steradian_angle
  return irradiance*10**(-9) # convert units from W/m^3 to W/(m^2*nm) to match units used on pvlighthouse website

In [46]:
wavelength = linspace(100, 3000, num = 2900) # in nanometers
irradiance = []

for i in wavelength:
  irradiance.append(irradiance_wavelength(Lambda = i * 10 ** (-9)))

irradiance_wavelength_df = DataFrame({'Wavelength(nm)': wavelength, 'Irradiance(W/(m^2 nm))': irradiance})

scatter(irradiance_wavelength_df, x = "Wavelength(nm)", y = 'Irradiance(W/(m^2 nm))')

B. The spectral irradiance versus photon energy

Frequency form of Planck's radiation law is:
$B_f(f, T) = \frac{2hf^3}{c^2(\exp(\frac{hf}{k_b T}) - 1)}$
To rewrite in terms of energy, we can use $E=hf$ to replace the $f$'s with $\frac{E}{h}$, and then multiply the whole thing by $\frac{\partial f}{\partial E} = \frac{1}{h}$
 That gives $B_E(E, T) = \frac{2\frac{E^3}{h^3}}{c^2(\exp(\frac{E}{k_b T}) - 1)} = \frac{2E^3}{h^3c^2(\exp(\frac{E}{k_b T}) - 1)}$


In [39]:
def irradiance_energy(E):
  h = qm.h
  k_b = qm.k_b
  return (2 * E**3) / ((exp(E / (k_b * T)) - 1) * c**2 * h**3) * sun_steradian_angle * q
  # times q to convert units from (eV/s)/(m^2 eV) to (J/s)/(m^2 eV) = W/(m^2 eV)

In [40]:
energy = linspace(0.01, 5, num = 2900) # in ev
irradiance = []

for i in energy:
  irradiance.append(irradiance_energy(E = i))

irradiance_energy_df = DataFrame({'Energy(eV)': energy, 'Irradiance(W/(eV m^2))': irradiance})

scatter(irradiance_energy_df, x = "Energy(eV)", y = 'Irradiance(W/(eV m^2))')

C. and D. The photon flux versus photon wavelength and versus energy

Let photon flux be denoted by $\phi$ and power by $P$.

As worked out in HW 1 problem 2, to convert from $P$ to $\frac{\text{# of photons}}{\text{s}}$, the conversion factor is $\frac{\lambda}{hc}$.

The units of $\phi_\lambda$ are $\frac{\text{# of photons}}{\text{s}\cdot\text{nm}\cdot \text{m}^2}$ and the units of $\phi_E$ are $\frac{\text{# of photons}}{\text{s}\cdot\text{eV}\cdot \text{m}^2}$.  The only difference in the corresponding units for irradiance is that the numerator has units of $P$ instead of $\frac{\text{# of photons}}{\text{s}}$.

 Hence, to convert from irradiance to flux in both cases, we simply multiply by $\frac{\lambda}{hc}$, which gives:
<br>$\phi_\lambda = B_\lambda  \frac{\lambda}{hc}$

For $\phi_E$, we should eliminate $\lambda$ from the conversion factor. We can do that by using $E=hf=\frac{hc}{\lambda} \implies \lambda=\frac{hc}{E} \implies \frac{\lambda}{hc} = \frac{\frac{hc}{E}}{hc}=\frac{1}{E}$

Hence, $\phi_E = B_E \frac{1}{E}$

Photon flux versus photon wavelength:

In [73]:
flux_wavelength_df = irradiance_wavelength_df.\
  rename(columns = {'Irradiance(W/(m^2 nm))': 'Photon Flux(# of photons/(s m^2 nm))'})

flux_wavelength_df['Photon Flux(# of photons/(s m^2 nm))'] = \
  flux_wavelength_df['Photon Flux(# of photons/(s m^2 nm))']*flux_wavelength_df['Wavelength(nm)']*1/(q*qm.h*c)
#deviding by q because because there's an extra multiplication by q in the irradiance_wavelength function so as to make the units for that have SI units in the numerator

scatter(flux_wavelength_df, x = "Wavelength(nm)", y = 'Photon Flux(# of photons/(s m^2 nm))')


Photon flux versus photon energy:


In [74]:
flux_energy_df = irradiance_energy_df.\
  rename(columns = {'Irradiance(W/(eV m^2))': 'Photon Flux(# of photons/(s m^2 eV))'})

flux_energy_df['Photon Flux(# of photons/(s m^2 eV))'] = \
  flux_energy_df['Photon Flux(# of photons/(s m^2 eV))']*1/flux_energy_df['Energy(eV)']*(1/q)
#deviding by q because because there's an extra multiplication by q in the irradiance_energy function so as to make the units for that have SI units in the numerator

scatter(flux_energy_df, x = "Energy(eV)", y = 'Photon Flux(# of photons/(s m^2 eV))')


# Task 2 (40 points):
Calculate the integral of the spectral irradiance over wavelength and compare it to the integral over of the spectral irradiance photon energy (first two plots). Both integrals should be identical. They can differ slightly based on the integration limits (wavelength or energy range). Repeat the integral calculations for the plots of the photon density, multiplying the photon density for each data point with the respective photon energy. The integrals should match the values for the previous cases. This is a sanity check whether the transformations were performed properly.
Since we are not taking absorption in the Earth atmosphere into account, the spectrum will be close to that of AM0, which has an integral value of 1366 W/m2. Since your integration limits are different, the value will be closer to 1300-1320 W/m2. You should make sure that your integration limits match, because that could lead to discrepancies and might cause confusion. You can calculate the integral on a different x-axis range than the plot if you want to “zoom in” on one region of the plot, nut you don’t have to. Make sure that the x-axis has points that are equidistantly spaced and have a delta of 1 between the points if you want to use the short form of the trapz function in numpy.
Task

In [87]:
from numpy import trapz

temp1 = trapz(irradiance_wavelength_df.loc[:, "Irradiance(W/(m^2 nm))"])
print('Area under curve of B_lambda = ', round(temp1, 2),"W/m^2")

temp2 = trapz(irradiance_energy_df.loc[:, "Irradiance(W/(eV m^2))"], irradiance_energy_df.loc[:, 'Energy(eV)'])
print('Area under curve of B_E = ', round(temp2, 2), "W/m^2")

print('\n')

#undo transformations applied to irradiance data to get flux data and integrate
temp1 = trapz(flux_wavelength_df.loc[:, "Photon Flux(# of photons/(s m^2 nm))"]*(q*qm.h*c)/flux_wavelength_df['Wavelength(nm)'])
print('Area under curve of phi_lambda = ', round(temp1, 2),"W/m^2")

temp2 = trapz(flux_energy_df.loc[:, 'Photon Flux(# of photons/(s m^2 eV))']*flux_energy_df['Energy(eV)']*q,
             flux_energy_df.loc[:, 'Energy(eV)'])
print('Area under curve of phi_E = ', round(temp2, 2), "W/m^2")

power_density = (temp1 + temp2)/2

Area under curve of B_lambda =  1341.48 W/m^2
Area under curve of B_E =  1358.22 W/m^2


Area under curve of phi_lambda =  1341.48 W/m^2
Area under curve of phi_E =  1358.22 W/m^2


# Task 3 (20 points):
Calculate the harvested power depending on the band gap of the absorber material.
You can use the photon energy to do “double duty”:
You can calculate the integral for the photon density versus energy plot for every photon energy point,
starting the integral at that photon energy and multiplying the photon density integral by that photon energy.
Once you have this data set, you can divide it by the value you obtained in Task 2 to get the efficiency.
Finally, plot the efficiency versus photon (or band gap) energy. There should be a peak around 1.2 eV.

In [112]:
solar_power_at_different_photon_energies = []

bandgap_energy = linspace(0.6, 5, num = 2900)  #in eV

for i in range(0, 2900):
	#take integral of
  x = flux_energy_df[flux_energy_df['Energy(eV)'] >= bandgap_energy[i]] ['Energy(eV)']

  temp = trapz(flux_energy_df[
           flux_energy_df['Energy(eV)'] >= bandgap_energy[i]]["Photon Flux(# of photons/(s m^2 eV))"]*bandgap_energy[i]*q, x)

  solar_power_at_different_photon_energies.append(temp)

efficiency = solar_power_at_different_photon_energies / power_density

efficiency_power_df = DataFrame({'efficiency(%)': efficiency * 100, "bandgap(eV)": bandgap_energy})

scatter(efficiency_power_df, y = 'efficiency(%)', x = "bandgap(eV)")


# Bonus (10 points):
Compare the efficiency of a single junction Silicon cell (Band gap 1.1eV) with a Perovskite Tandem cell (1.2eV and 1.8eV). For the silicon cell, you can just pick a value out of your array or start the integration again at 1.1eV, multiplying with 1.1eV band gap energy. For the Tandem cell, you need to stop the integration at the larger band gap value and then re-start the integration there, but multiply with the larger band gap value.

In [138]:
photon_flux: str = "Photon Flux(# of photons/(s m^2 eV))"

def efficiency(bandgap):
  x = flux_energy_df[flux_energy_df['Energy(eV)'] >= bandgap] ['Energy(eV)']
  harvested_power = trapz(flux_energy_df[flux_energy_df['Energy(eV)'] >= bandgap][photon_flux]*bandgap*q, x)
  return harvested_power/power_density

def tandem_efficiency(gap1, gap2):
  energy = flux_energy_df['Energy(eV)']

  x1 = flux_energy_df[(energy >= gap1) & (energy <= gap2)]['Energy(eV)']
  x2 = flux_energy_df[energy >= gap2]['Energy(eV)']

  harvested_power1 = trapz(flux_energy_df[(energy >= gap1) & (energy <= gap2)][photon_flux]*gap1*q, x1)
  harvested_power2 = trapz(flux_energy_df[flux_energy_df['Energy(eV)'] >= gap2][photon_flux]*gap2*q, x2)

  return (harvested_power1 + harvested_power2)/power_density

print("Efficiency of silicon solar cell = ",  efficiency(1.1))
print("Efficiency of Perovskite Tandem cell = ",  tandem_efficiency(1.2, 1.8))

Efficiency of silicon solar cell =  0.4430803313235519
Efficiency of Perovskite Tandem cell =  0.5516563015827783
